# ollama 실습
* 다운로드 링크: https://github.com/ollama/ollama
* 사용한 모델: llama3


In [1]:
import ollama

In [2]:
response = ollama.chat(model="llama3", messages=[{"role": "user", "content": "Hello, how are you?"}])
print(response['message']['content'])

I'm just an AI, so I don't have feelings like humans do. But I'm here to help with any questions or topics you'd like to discuss! How about you, though - how's your day going?


In [3]:
response = ollama.chat(model="llama3", messages=[{"role": "user", "content": "안녕"}])
print(response['message']['content'])

😊 안녕하세요! (Annyeonghaseyo!) How can I help you today? 😊


# OpenAI

선행 작업
* 키 설정
```
!export OPENAI_API_KEY="your_api_key_here"
```
* 패키지 다운
```
!pip install openai
```

In [4]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Code calls itself back,  \nEach layer unveils the path,  \nEndless loops of thought.', role='assistant', function_call=None, tool_calls=None, refusal=None, annotations=[])


In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "안녕!"
        }
    ]
)

# 모델의 응답 출력
print(response.choices[0].message.content)

안녕하세요! 어떻게 도와드릴까요?


## Prompt Engineering

Self attention 구조를 고려한 prompting 를 작성하기

예시) best place 보다 tourist atrraction 이 질문의 의도에 더 명확하게 부합하는 embedding 값을 제공할 것
```
What are the best places in Korea?
> What are the tourist attractions in Korea?
```

### Few shot engineering

1. 영어를 한국어로 번역하는 5-shot prompt 를 작성하고 "dog"를 번역해보세요

In [6]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "영어가 들어오면 한글로 번역해줘 \n ex) {input: cat, output: 고양이}, {input: book, output: 책}, {input: computer output: 컴퓨터}, {input: coffee, output: 커피}, {input: water, output: 물}"
        },
        {
            "role": "user",
            "content": "dog"
        }
    ]
)

# 모델의 응답 출력
print(response.choices[0].message.content)

강아지


2. 영화 리뷰에 대한 sentiment (positive or negative) 를 결정하는 5-shot prompt 를 작성하고 "The storyline was dull and uninspiring." 에 대한 결과를 확인해보세요.

In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "리뷰가 입력되면 positive, negative로 분류해줘 \n ex) {input: 재미없다, output: negative}, {input: 시간 가는 줄 몰랐어요, output: positive}, {input: 다시는 안 볼 듯 output: negative}, {input: 믿고 보는 감독 작품, output: positive}, {input: 돈 아깝다, output: negative}"
        },
        {
            "role": "user",
            "content": "The storyline was dull and uninspiring"
        }
    ]
)

# 모델의 응답 출력
print(response.choices[0].message.content)

{input: The storyline was dull and uninspiring, output: negative}


3. 자연어를 SQL 쿼리로 바꿔주는 few-shot prompt 를 작성해보세요.

In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '''
Convert the following natural language requests into SQL queries:
1. "급여가 50,000 이상인 모든 직원 정보를 보여주세요.": SELECT * FROM employees WHERE salary > 50000;
2. "재고가 0인 모든 제품 정보를 보여주세요.": SELECT * FROM products WHERE stock = 0;
3. "수학 점수가 90점 이상인 학생들의 이름을 나열해주세요.": SELECT name FROM students WHERE math_score > 90;
4. "최근 30일 이내에 주문된 모든 주문 정보를 보여주세요.": SELECT * FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);
5. "도시별 고객 수를 보여주세요.": SELECT city, COUNT(*) FROM customers GROUP BY city;
            '''
        },
        {
            "role": "user",
            "content": "Find the average salary of employees in the marketing department."
        }
    ]
)

# 모델의 응답 출력
print(response.choices[0].message.content)

Here is the SQL query to find the average salary of employees in the marketing department:

```sql
SELECT AVG(salary) FROM employees WHERE department = 'Marketing';
```


### Chain of Thought

In [9]:
# Simple
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '''
# Simple - 1
Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. 일의 자리 숫자끼리 더한다. 3 + 7 = 10
2. 십의 자리 숫자끼리 더한다. 20 + 40 = 60
3. 1번과 2번의 결과를 더한다. 10 + 60 = 70

Answer: 70

# Simple - 2
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. 백의 자리 숫자끼리 뺀다. 1 - 0 = 1
2. 십의 자리 숫자끼리 뺀다. 2 - 5 (x)
3. 2가 5보다 작아 뺄셈이 불가능하므로 백의 자리 숫자 1을 가져온다. 12 - 5 = 7 또한 백의 자리에서 1을 뺀다. 1 - 1 = 0
4. 일의 자리 숫자끼리 뺀다. 3 - 8 (x)
5. 3이 8보다 작아 뺄셈이 불가능하므로 십의 자리에서 1을 가져온다. 13 - 8 = 5 또한 십의 자리에서 1을 뺀다. 7 - 1 = 6

Answer: 65
            '''
        },
        {
            "role": "user",
            "content": "Solve the following problem step-by-step: 345 + 678 - 123"
        }
    ]
)

# 모델의 응답 출력
print(response.choices[0].message.content)

Let's solve the problem step-by-step: 345 + 678 - 123.

First, we will perform the addition (345 + 678).

### Step 1: Addition
1. 일의 자리 숫자끼리 더한다: 5 + 8 = 13 (일의 자리 수는 3, 1은 올림)
2. 십의 자리 숫자끼리 더한다: 4 + 7 + 1 (올림) = 12 (십의 자리 수는 2, 1은 올림)
3. 백의 자리 숫자끼리 더한다: 3 + 6 + 1 (올림) = 10 (백의 자리 수는 0, 1은 올림)

결과: 1023

### Step 2: Subtraction
Now, we need to subtract 123 from 1023.

1. 일의 자리 숫자끼리 뺀다: 3 - 3 = 0
2. 십의 자리 숫자끼리 뺀다: 2 - 2 = 0
3. 백의 자리 숫자끼리 뺀다: 10 - 1 = 9 (여기서 10은 올림을 고려한 결과입니다)

결과: 900

### Final Answer:
900


In [17]:
# Simple
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '''
# Intermediate - 1
Solve the following logic puzzle step-by-step:
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Determine the favorite color of each friend.

Step-by-step solution:
1. Carol은 green을 좋아한다. 따라서 red, blue color가 남았다.
2. Alice는 red를 싫어하므로 blue를 좋아한다.
3. Bob은 마지막으로 남은 색인 red를 좋아한다.

Answer:
- Alice: blue
- Bob: red
- Carol: green
            '''
        },
        {
            "role": "user",
            "content": '''
Four people (A, B, C, D) are sitting in a row. We know that:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Write Possible arrangements
            '''
        }
    ]
)

# 모델의 응답 출력
print(response.choices[0].message.content)

To solve this logic puzzle step-by-step, let's analyze the clues provided:

1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Let's denote the four positions in the row as 1, 2, 3, and 4. We'll look for possible arrangements based on the clues.

### Step-by-Step Analysis:

1. **B is next to C**:
   - This means B and C must be adjacent.
   - Possible pairs for B and C are (B, C) or (C, B). 

2. **C is not next to D**:
   - This means if C is at a certain position, D must not occupy the adjacent positions.

3. **A is not next to B**:
   - A cannot be in the positions directly adjacent to B.

### Possible Arrangements:
Let's analyze the arrangements based on the clues.

**Case 1: Arrangement with B next to C**

#### Pairing B and C as (B, C):
- If B is in position 1: 
  - Options: 
    - B (1), C (2), A (3), D (4)  -> Valid (A is not adjacent to B)
    - B (1), C (2), D (3), A (4)  -> Invalid (D is next to C)
  
- If B is in position 2: 
  - Options: 
    - A (1), B (2), 

## Prompt Engineering Evaluation 자동화 (Using Prompt Bench)

In [11]:
import promptbench as pb

In [12]:
# print all supported datasets in promptbench
print('All supported datasets: ')
print(pb.SUPPORTED_DATASETS)

# load a dataset, sst2, for instance.
# if the dataset is not available locally, it will be downloaded automatically.
dataset_name = "gsm8k"
dataset = pb.DatasetLoader.load_dataset(dataset_name)

# print the first 3 examples
dataset[:3]

All supported datasets: 
['sst2', 'cola', 'qqp', 'mnli', 'mnli_matched', 'mnli_mismatched', 'qnli', 'wnli', 'rte', 'mrpc', 'mmlu', 'squad_v2', 'un_multi', 'iwslt2017', 'math', 'bool_logic', 'valid_parentheses', 'gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking', 'last_letter_concat', 'numersense', 'qasc', 'bbh', 'drop', 'arc-easy', 'arc-challenge']


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

[{'content': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
  'label': '18'},
 {'content': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?',
  'label': '3'},
 {'content': 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?',
  'label': '70000'}]

In [13]:
# print all supported models in promptbench
print('All supported models: ')
print(pb.SUPPORTED_MODELS)

# load a model, gpt-3.5-turbo, for instance.
# If model is openai/palm, need to provide openai_key/palm_key
# If model is llama, vicuna, need to provide model dir
model = pb.LLMModel(model='gpt-3.5-turbo',
                    api_key = 'openai_key',
                    max_new_tokens=150)

All supported models: 
['google/flan-t5-large', 'llama2-7b', 'llama2-7b-chat', 'llama2-13b', 'llama2-13b-chat', 'llama2-70b', 'llama2-70b-chat', 'phi-1.5', 'phi-2', 'palm', 'gpt-3.5-turbo', 'gpt-4', 'gpt-4-1106-preview', 'gpt-3.5-turbo-1106', 'gpt-4-0125-preview', 'gpt-3.5-turbo-0125', 'gpt-4-turbo', 'gpt-4o', 'vicuna-7b', 'vicuna-13b', 'vicuna-13b-v1.3', 'google/flan-ul2', 'gemini-pro', 'mistralai/Mistral-7B-v0.1', 'mistralai/Mistral-7B-Instruct-v0.1', 'mistralai/Mixtral-8x7B-v0.1', 'mistralai/Mixtral-8x7B-Instruct-v0.1', '01-ai/Yi-6B', '01-ai/Yi-34B', '01-ai/Yi-6B-Chat', '01-ai/Yi-34B-Chat', 'baichuan-inc/Baichuan2-7B-Base', 'baichuan-inc/Baichuan2-13B-Base', 'baichuan-inc/Baichuan2-7B-Chat', 'baichuan-inc/Baichuan2-13B-Chat']


In [14]:
# load method
# print all methods and their supported datasets
print('All supported methods: ')
print(pb.SUPPORTED_METHODS)
print('Supported datasets for each method: ')
print(pb.METHOD_SUPPORT_DATASET)

# load a method, emotion_prompt, for instance.
# https://github.com/microsoft/promptbench/tree/main/promptbench/prompt_engineering
method = pb.PEMethod(method='emotion_prompt',
                        dataset=dataset_name,
                        verbose=True,  # if True, print the detailed prompt and response
                        prompt_id = 1  # for emotion_prompt
                        )

All supported methods: 
['CoT', 'ZSCoT', 'least_to_most', 'generated_knowledge', 'expert_prompting', 'emotion_prompt', 'baseline']
Supported datasets for each method: 
{'CoT': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'ZSCoT': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'expert_prompting': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'emotion_prompt': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'least_to_most': ['gsm8k', 'last_letter_concat'], 'generated_knowledge': ['csqa', 'numersense', 'qasc'], 'baseline': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking', 'last_letter_concat', 'numersense', 'qasc']}


In [16]:
results = method.test(dataset,
                      model,
                      num_samples=3 # if don't set the num_samples, method will use all examples in the dataset
                      )
results

  0%|                                                  | 0/1319 [00:00<?, ?it/s]


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: openai_key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# load method
# print all methods and their supported datasets
print('All supported methods: ')
print(pb.SUPPORTED_METHODS)
print('Supported datasets for each method: ')
print(pb.METHOD_SUPPORT_DATASET)

# load a method, emotion_prompt, for instance.
# https://github.com/microsoft/promptbench/tree/main/promptbench/prompt_engineering
method = pb.PEMethod(method='CoT',
                        dataset=dataset_name,
                        verbose=True,  # if True, print the detailed prompt and response
                        prompt_id = 1  # for emotion_prompt
                        )

In [ ]:
results = method.test(dataset,
                      model,
                      num_samples=3 # if don't set the num_samples, method will use all examples in the dataset
                      )
results

## Prompt Compression

In [18]:
from llmlingua import PromptCompressor

# llm_lingua = PromptCompressor(
#     model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
#     use_llmlingua2=True, # Whether to use llmlingua-2
# )

## Use LLMLingua-2-small model
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True, # Whether to use llmlingua-2
)


/usr/local/Caskroom/miniforge/base/envs/DL-torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/Caskroom/miniforge/base/envs/DL-torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
prompt = """Madam Court, could you please read docket 1239? Certainly. Docket 1239. The Committee on Government Operations, to which was referred on December 1st, 2021, docket number 1239 message an order authorizing the creation of a sheltered market program in conformity with the requirements of general laws. Chapter 30 B Section 18. This authorization applies to contracts for goods, professional services and support services. This authorization is for no more than six contracts, which must be awarded by June 30th, 2022. This sheltered market program shall be available for disadvantaged, minority and women only vendors, for whom there is a demonstrated substantial disparity in the city's 2020 disparities. Study submits a report recommending the order ought to pass. Thank you so much, Madam Clerk. The Chair recognizes Councilor Edwards, chair of the committee. Councilor Edwards. You have the floor. This is this is actually a matter, I believe, sponsored by the. Mayor in Cannes. In conformance with the recommendations from the disparity study and making sure that we opt in to this this pilot program under mass general laws 30 Section 18. Again, it's really just following the recommendations of an already studied issue, which which demonstrates a disparity between minority contractors or women contractors receiving contracts in the city of Boston. So this would allow for us to shepherd and move these six contracts to those already designated groups who have a disadvantage. And I think it's. Really fulfilling a promise. Of making sure that we go through and make sure all aspects of the city government, including the financial benefits, are accessible to people in the city of Boston. I recommend that this pass and I hope that my colleagues will vote for it. Thank you. Thank you so much. Councilor Edward seeks acceptance of the committee report and passage of Docket 1239. Madam Court, could you please call the roll? Certainly. Docket 1239. Councilor Arroyo. Yes. Councilor Arroyo. Yes. Councilor Baker. Councilor Baker. Councilor. Councilor Barker. Council Braden. Councilor Braden. Councilor Campbell. Councilor Campbell. Yes. Councilor Edwards. Yes. Councilor Sabby. George. Councilor Sabby. George. He has Councilor Flaherty. Councilor Flaherty as Councilor Flynn. Councilor Flynn. Yes. Councilor Jane. Yes. Councilor Janey. As Councilor me here. Councilor me here as Councilor Murphy. Councilor Murphy. Yes. And Councilor O'Malley. Yes. Councilor O'Malley. Yes. Madam President, do I get number 1239 has received unanimous vote. Thank you so much. Dockett 1239 has passed and now we will move on to matters recently heard for possible action. Madam Clerk, if you could please read docket 0863. Certainly Docket 0863 order for hearing to discuss pest control and illegal dumping in the city of Boston."""
compressed_prompt = llm_lingua.compress_prompt(prompt, rate=0.33, force_tokens = ['\n', '?'])

In [ ]:
print('original prompt length: ', len(prompt))
print('compressed prompt length: ',len(compressed_prompt['compressed_prompt']))
compressed_prompt['compressed_prompt']